In [10]:
import numpy as np
import os, sys
sys.path.append(os.path.abspath(os.path.join('../../..')))
import tensorflow as tf

from scipy import integrate as spi
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

In [11]:
current_dir = os.getcwd()
print("Current working directory: {0}".format(os.getcwd()))
os.chdir('/home/rene/BayesFlow') #Location of BayesFlow on hard drive

from bayesflow.forward_inference import GenerativeModel, Prior, Simulator
from bayesflow.networks import InvertibleNetwork, InvariantNetwork
from bayesflow.amortized_inference import AmortizedPosterior
from bayesflow.trainers import Trainer

os.chdir(current_dir)

Current working directory: /home/rene/BayesFlow


In [6]:
A = np.asarray([1.2,-5.0,30.0,-7.5, 0.75])
B = np.asarray([0.25, 0.1, 0.1, 0.1, 0.4])
Phi = np.pi*np.asarray([-1/3,-1/12,0,1/12,0.5])
#Phi = (np.pi / 180) * np.asarray([-70, -15, 0, 15, 100])

A_1 = np.asarray([1.2,-5.0,30.0,-7.5, 0.75])
B_1 = np.asarray([0.25, 0.1, 0.1, 0.1, 0.6])
Phi_1 = np.pi*np.asarray([-1/3,-1/12,0,1/12,0.5])

params_ODE = {'a': A, 'b': B, 'phi': Phi}
params_ODE_1 = {'a': A_1, 'b': B_1, 'phi': Phi_1}

Nt = 13000
f_s = 1000
f_2 = 0.25
#pos_0 = np.asarray([1, 0, 0.04])
pos_0 = np.asarray([1, 0, 0])


max_t = (Nt-1)*(1/f_s)
params_sim = {'f_2': f_2, 'f_s': f_s, 'Nt': Nt, 'max_t': max_t, 'pos_0': pos_0, 'Tspan': [0, max_t], \
             't_eval': np.linspace(0, max_t, Nt)}

# f_1 = 0.1
# c_1 = 0.01
# c_2 = 0.01
# sigma_1 = 1

# params_TS = {'f_1': f_1, 'f_2': f_2, 'c_1': c_1, 'c_2': c_2, \
#              'sigma_1': sigma_1, 'sigma_2': (2 * sigma_1) ** 0.5}

print("\n Params ODE:")
print(params_ODE)
# print("\n Params Time Series:")
# print(params_TS)
print("\n Params simulation:")
print(params_sim)


 Params ODE:
{'a': array([ 1.2 , -5.  , 30.  , -7.5 ,  0.75]), 'b': array([0.25, 0.1 , 0.1 , 0.1 , 0.4 ]), 'phi': array([-1.04719755, -0.26179939,  0.        ,  0.26179939,  1.57079633])}

 Params simulation:
{'f_s': 1000, 'Nt': 13000, 'max_t': 12.999, 'pos_0': array([1, 0, 0]), 'Tspan': [0, 12.999], 't_eval': array([0.0000e+00, 1.0000e-03, 2.0000e-03, ..., 1.2997e+01, 1.2998e+01,
       1.2999e+01])}


In [9]:
hrmean = 480
hrfact = np.sqrt(hrmean / 60)
hrfact2 = np.sqrt(hrfact)
Phi_ext = np.array([hrfact2, hrfact, 1, hrfact, hrfact2]) * Phi
print(Phi_ext/np.pi)

[-0.56059761 -0.23570226  0.          0.23570226  0.84089642]


In [ ]:
def alpha(x,y):
    return(1-(x**2 + y**2)** .5)

def delta_phi(x,y,phi):
    return((np.arctan2(y,x)-phi) % (2*np.pi))

def gauss_sum(x,y,a,b,phi):
    gauss_summands = -a*delta_phi(x,y,phi)*np.exp(-(delta_phi(x,y,phi)**2)/(2*(b**2)))
    return(np.sum(gauss_summands))

def z_0(t,f_2):
    return(0.15*np.sin(2*np.pi*f_2*t))

def omega(t, T, max_t, Nt):
    t_index = int((t/max_t*Nt)-1)
    return(np.pi*2/T[t_index])

def ECG_model(t, p, T, params_ODE, params_sim, baseline_wander=False, time_dependent_omega=False):
    if baseline_wander:
        z_zero = z_0
    else:
        z_zero = lambda t, f_2: 0
    
    position = [alpha(p[0],p[1])*p[0]-p[1], alpha(p[0],p[1])*p[1]+p[0], \
                gauss_sum(p[0],p[1],params_ODE['a'],params_ODE['b'],params_ODE['phi'])-p[2]+z_zero(t,f_2)]
    
    return(position)

def generate_ECG(f_s, pos_0, params_ODE, params_sim, \
                 baseline_wander=False, time_dependent_omega=False, add_noise=False):
    
    T = generate_time_series(params_TS, params_sim, 0, 1)
    
    positions = spi.solve_ivp(lambda t, y: \
                  ECG_model(t, y, T, params_ODE, params_TS, baseline_wander, time_dependent_omega), \
                  params_sim['Tspan'], params_sim['pos_0'], t_eval=params_sim['t_eval']).y
    
    if add_noise:
        magnitude = np.mean(positions[2])
        positions[2] += 10*(2 * np.random.uniform(len(positions[2])) - 1)
    return(positions)
